In [65]:
import os
import json
import pandas as pd
import numpy as np

import ast

import pickle 
import matplotlib.pyplot as plt 

from sklearn.feature_selection import *
from sklearn.model_selection import *
from sklearn.preprocessing import *
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline

from sklearn.svm import SVC, SVR, LinearSVC
from sklearn.metrics import *
from sklearn.model_selection import cross_val_score

from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier


from sklearn.base import TransformerMixin
from sklearn.metrics import *

In [2]:
X = np.loadtxt("data/processed/filled.np.gz")

In [53]:
with open("data/processed/processed_data.json") as jfile:
    df = json.load(jfile)["df"]
    df = pd.DataFrame(df)

print(df)

      Age Gender Status                                           features
0      28      F      H  [-22.657711933297467, 30.769295879945037, -23....
1      24      M      S  [182.05834145671884, 20.138809568394194, -6.23...
2      29      M      N  [-21.308495839973528, -12.517586239957701, -9....
3      28      M      H  [-22.320687052787786, 22.945827273719978, -22....
4      25      M      N  [-22.65158385249504, 25.13058078215149, -11.95...
...   ...    ...    ...                                                ...
2600   21      F      N  [-21.756444948496828, -33.34037201123824, -2.2...
2601   24      F      H  [-20.832185096880092, 1.4025807238407926, -12....
2602   12      F      H  [-20.441490886122775, 14.783729334551827, -7.3...
2603   25      M      H  [-20.757076001239067, -24.569769281532093, -1....
2604    6      M      H  [-20.658551689088036, 10.331795410271129, 40.6...

[2545 rows x 4 columns]


In [83]:
with open("data/processed/indices.json") as jfile:
    indices = json.load(jfile)


# We created a set of indices for each subset, and it's important to keep 
# Test indices seperate from our hyperparameter tunings,
# Only validation is tested for feature selection and etc

train_indices = np.array(indices["train_indices"])
val_indices = np.array(indices["val_indices"])
test_indices = np.array(indices["test_indices"])

X_train = X[train_indices]
X_val = X[val_indices]
X_test = X[test_indices]


feature_dims = np.array(indices["feature_dims"])
features = "f1 f2 f3 f4 f5".split()


# Feature Ranking

In [81]:
gender_encoder = LabelEncoder().fit(df["Gender"])
status_encoder = LabelEncoder().fit(df["Status"])

genders = gender_encoder.transform(df["Gender"])
status = status_encoder.transform(df["Status"])

gender_onehot = OneHotEncoder().fit_transform(genders.reshape(-1, 1))
status_onehot = OneHotEncoder().fit_transform(status.reshape(-1, 1))


In [82]:
# Boundaries in the X matrix, each feature set starts from a start_column and ends in a end_column
feature_boundaries = {}
start_idx = 0
for f, feature_len in zip(features, feature_dims):
    feature_boundaries[f] = (start_idx, start_idx + feature_len)
    start_idx += feature_len
feature_boundaries

{'f1': (0, 512),
 'f2': (512, 2048),
 'f3': (2048, 4352),
 'f4': (4352, 6912),
 'f5': (6912, 7116)}

## Feature Set Importance Using Random Forests


We use a random forest clf to classify the most important features for each label, starting with age.

First we calculate a "leave feature out" error rate, to compare, then we calculate direct feature importances on a PCA projected set of each original set, and show some metrics. 

First we define some custom helper classes.

In [70]:
# Defining a custom pca for cross validation

class CustomPCA(TransformerMixin):
    def __init__(self, indices, explained_var=0.8):
        super().__init__()
        self.explained_var = explained_var
        self.indices = indices
        self.pcas = {}

    def fit(self, X, y=None, **kwargs):
        for k in self.indices:
            start_col, end_col = self.indices[k]
            self.pcas[k] = Pipeline(
                [
                    ("scaler", StandardScaler()),
                    ("pca", PCA(self.explained_var))
                ]
            )
            self.pcas[k].fit_transform(X[:, start_col:end_col])
        return self
        
    def transform(self, X, **kwargs):
        X_transform = []
        for k in self.indices:
            start_col, end_col = self.indices[k]
            X_transform.append(self.pcas[k].transform(X[:, start_col:end_col]))
        return np.concatenate(X_transform, axis=-1)

In [67]:
class ProbaScorer:
    def __init__(self):
        pass

    def __call__(self, clf, X, y):
        '''
            calls predict_proba function of clf and computes
            a bayesian mse for the clf
        '''
        probs = clf.predict_proba(X)
        n_class = len(np.unique(y))
        one_hot = np.zeros((len(X), n_class))
        one_hot[:, y] = 1
        mse = ((probs - one_hot) ** 2).mean()
        return mse

In [84]:
class FeatureImportanceScorer:
    def __init__(self, indices):
        self.indices = indices
    
    def __call__(self, clf, X=None, y=None):
        '''
            reporting mean, max and std of importances of random forest
        '''
        result = []
        for f in self.indices:
            start_col, end_col = self.indices[f]
            importances = clf.feature_importances_[start_col:end_col]
            result.append({
                "feature set": f,
                "max_importance": importances.max(),
                "mean_importance": importances.mean(),
                "std_importance": importances.std(),
            })
        return result


In [86]:
# Projecting from high dimensional data to each feature set's PC space
pca = CustomPCA(feature_boundaries).fit(X[train_indices])
X_train_transform = pca.transform(X_train)
X_val_transform = pca.transform(X_val)

In [93]:
# The previous indices does not work, we need to find the new indices for each pc set
transformed_feature_boundaries = {}
start = 0
for k in pca.pcas:
    end = start + pca.pcas[k]["pca"].n_components_
    transformed_feature_boundaries[k] = (start, end)
    start = end
transformed_feature_boundaries

{'f1': (0, 27), 'f2': (27, 63), 'f3': (63, 88), 'f4': (88, 95), 'f5': (95, 98)}

#### Gender Importances

In [106]:
gender_info = []

clf = RandomForestClassifier(n_estimators=100, )

for f in features:
    # all sets except one
    indices = [i for k, (start, end) in transformed_feature_boundaries.items() for i in range(start, end) if k != f]
    curr_X_train = X_train_transform[..., indices]
    curr_X_val = X_val_transform[..., indices]
    clf.fit(curr_X_train, genders[train_indices])
    preds = clf.predict(curr_X_val)
    target = genders[val_indices]
    gender_info.append(
        {
            "except_feature": f,
            "accuracy": accuracy_score(preds, target),
            "bayes_mse": ProbaScorer()(clf, curr_X_train, target),
        }
    )


print("####\n#### Gender ####\n####")

print(f"Error rate for leave one feature out:\n\n{pd.DataFrame(gender_info)}\n\n")

clf.fit(X_train_transform, genders[train_indices])
gender_set_importances = FeatureImportanceScorer(transformed_feature_boundaries)(clf)

print(f"Collective Feature Importances:\n\n{pd.DataFrame(gender_set_importances)}\n\n")

####
#### Gender ####
####
Error rate for leave one feature out:

  except_feature  accuracy  bayes_mse
0             f1  0.945098   0.433745
1             f2  0.972549   0.446248
2             f3  0.968627   0.446806
3             f4  0.980392   0.446873
4             f5  0.976471   0.446447


Collective Feature Importances:

  feature set  max_importance  mean_importance  std_importance
0          f1        0.214381         0.013744        0.039419
1          f2        0.097156         0.008957        0.017545
2          f3        0.086250         0.009820        0.016765
3          f4        0.005190         0.003885        0.000748
4          f5        0.018484         0.011253        0.005418




#### Status Importances

In [107]:
status_info = []

clf = RandomForestClassifier(n_estimators=100, )

for f in features:
    # all sets except one
    indices = [i for k, (start, end) in transformed_feature_boundaries.items() for i in range(start, end) if k != f]
    curr_X_train = X_train_transform[..., indices]
    curr_X_val = X_val_transform[..., indices]
    clf.fit(curr_X_train, status[train_indices])
    preds = clf.predict(curr_X_val)
    target = status[val_indices]
    status_info.append(
        {
            "except_feature": f,
            "accuracy": accuracy_score(preds, target),
            "bayes_mse": ProbaScorer()(clf, curr_X_train, target),
        }
    )


print("####\n#### STATUS ####\n####")

print(f"Error rate for leave one feature out:\n\n{pd.DataFrame(status_info)}\n\n")

clf.fit(X_train_transform, status[train_indices])
status_set_importances = FeatureImportanceScorer(transformed_feature_boundaries)(clf)

print(f"Collective Feature Importances:\n\n{pd.DataFrame(status_set_importances)}\n\n")

####
#### STATUS ####
####
Error rate for leave one feature out:

  except_feature  accuracy  bayes_mse
0             f1  0.376471   0.541448
1             f2  0.325490   0.541568
2             f3  0.364706   0.541999
3             f4  0.349020   0.542433
4             f5  0.384314   0.542347


Collective Feature Importances:

  feature set  max_importance  mean_importance  std_importance
0          f1        0.011925         0.010428        0.000597
1          f2        0.011913         0.010126        0.000971
2          f3        0.012903         0.010057        0.001272
3          f4        0.011911         0.010374        0.001031
4          f5        0.010364         0.009958        0.000402




In [108]:
regressor = RandomForestRegressor(n_estimators=100)

age = df["Age"].to_numpy()

age_info = []

for f in features:
    # all sets except one
    indices = [i for k, (start, end) in transformed_feature_boundaries.items() for i in range(start, end) if k != f]
    curr_X_train = X_train_transform[..., indices]
    curr_X_val = X_val_transform[..., indices]
    regressor.fit(curr_X_train, age[train_indices])
    preds = regressor.predict(curr_X_val)
    target = age[val_indices]
    age_info.append(
        {
            "except_feature": f,
            "mse": mean_squared_error(preds, target),
            "mae": mean_absolute_error(preds, target),
        }
    )


print("####\n#### AGE ####\n####")

print(f"Error rate for leave one feature out:\n\n{pd.DataFrame(age_info)}\n\n")

regressor.fit(X_train_transform, age[train_indices])

age_set_importances = FeatureImportanceScorer(transformed_feature_boundaries)(regressor)

print(f"Collective Feature Importances:\n\n{pd.DataFrame(age_set_importances)}\n\n")

####
#### AGE ####
####
Error rate for leave one feature out:

  except_feature        mse       mae
0             f1  52.454019  4.500816
1             f2  40.658732  3.682593
2             f3  40.274289  3.661965
3             f4  35.812158  3.510609
4             f5  37.810984  3.643460


fitted
Collective Feature Importances:

  feature set  max_importance  mean_importance  std_importance
0          f1        0.240749         0.016119        0.044558
1          f2        0.135947         0.011025        0.026809
2          f3        0.043462         0.005574        0.008669
3          f4        0.003905         0.002565        0.000651
4          f5        0.005884         0.003536        0.001664


